Read the sales data Excel sheet. There are 4 rows preceeding the header

In [33]:
import pandas as pd

sales = pd.read_excel('../data/original/rollingsales_brooklyn.xls', header=4)
sales

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6359,70,NaN,S1,8684 15TH AVENUE,NaN,...,1.0,1.0,2.0,1933.0,4080.0,1930.0,1,S1,1300000,2020-04-28
1,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6360,48,NaN,A5,14 BAY 10TH STREET,NaN,...,1.0,0.0,1.0,2513.0,1428.0,1930.0,1,A5,849000,2020-03-18
2,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6367,24,NaN,A9,8645 BAY 16TH STREET,NaN,...,1.0,0.0,1.0,1571.0,1456.0,1935.0,1,A9,830000,2020-06-26
3,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6371,19,NaN,A9,79 BAY 20TH STREET,NaN,...,1.0,0.0,1.0,2320.0,1566.0,1930.0,1,A9,0,2020-02-21
4,3,BATH BEACH,01 ONE FAMILY DWELLINGS,1,6371,60,NaN,A9,8620 19TH AVENUE,NaN,...,1.0,0.0,1.0,2417.0,2106.0,1930.0,1,A9,1188000,2020-07-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18593,3,WYCKOFF HEIGHTS,30 WAREHOUSES,4,3345,6,NaN,E1,333 IRVING AVENUE,NaN,...,0.0,1.0,1.0,7450.0,1950.0,1952.0,4,E1,11500000,2019-11-12
18594,3,WYCKOFF HEIGHTS,41 TAX CLASS 4 - OTHER,4,3407,21,NaN,Z9,376 WEIRFIELD STREET,NaN,...,0.0,0.0,0.0,10058.0,0.0,NaN,4,Z9,0,2020-08-19
18595,3,WYCKOFF HEIGHTS,44 CONDO PARKING,4,3328,1086,NaN,RP,"358 GROVE ST, P25",P25,...,NaN,1.0,1.0,NaN,NaN,NaN,4,RP,1,2020-03-09
18596,3,WYCKOFF HEIGHTS,44 CONDO PARKING,4,3328,1093,NaN,RP,"358 GROVE STREET, P32",P32,...,NaN,1.0,1.0,NaN,NaN,NaN,4,RP,618800,2019-10-15


Filter out 
- garages, offices, etc (keep only familiy dwelling) 
- entries with price of $10 and below (assuming if one gets a dwelling gifted, this buyer guide is not used)
- entries with gross square feet of zero

In [34]:
building_classes = ["01 ONE FAMILY DWELLINGS", "02 TWO FAMILY DWELLINGS", "03 THREE FAMILY DWELLINGS"]

sales = sales[(sales["BUILDING CLASS CATEGORY"].isin(building_classes)) & (sales["SALE PRICE"] > 10) & (sales["GROSS SQUARE FEET"] > 0)].copy()

Get rid of unused columns

In [36]:
sales.drop(columns=["BOROUGH","NEIGHBORHOOD","TAX CLASS AT PRESENT","BLOCK","LOT","EASE-MENT","BUILDING CLASS AT PRESENT","APARTMENT NUMBER","TOTAL UNITS","TAX CLASS AT TIME OF SALE","BUILDING CLASS AT TIME OF SALE","SALE DATE", "RESIDENTIAL UNITS","COMMERCIAL UNITS","ADDRESS","LAND SQUARE FEET", "YEAR BUILT", "BUILDING CLASS CATEGORY"], inplace=True)

sales

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE,PRICE PER GSQ
0,11228,4080.0,1300000,318.627451
1,11228,1428.0,849000,594.537815
2,11214,1456.0,830000,570.054945
4,11214,2106.0,1188000,564.102564
5,11228,1804.0,975000,540.465632
...,...,...,...,...
18529,11237,2864.0,1062500,370.984637
18532,11237,3084.0,900000,291.828794
18541,11237,2700.0,1035000,383.333333
18542,11237,2700.0,980000,362.962963


Calculate a new column: Price per gross square feet
(see https://home.akitabox.com/blog/gross-square-footage-definitions)

In [37]:
sales["PRICE PER GSQ"] = sales["SALE PRICE"] / sales["GROSS SQUARE FEET"]
sales

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE,PRICE PER GSQ
0,11228,4080.0,1300000,318.627451
1,11228,1428.0,849000,594.537815
2,11214,1456.0,830000,570.054945
4,11214,2106.0,1188000,564.102564
5,11228,1804.0,975000,540.465632
...,...,...,...,...
18529,11237,2864.0,1062500,370.984637
18532,11237,3084.0,900000,291.828794
18541,11237,2700.0,1035000,383.333333
18542,11237,2700.0,980000,362.962963


group the sales by ZIP CODE. And then index must be removed, otherwise Folium/Colorpleth gets into troubles later

In [50]:
sales_grouped = sales.groupby(['ZIP CODE']).mean()
sales_grouped = sales_grouped.reset_index()
sales_grouped.to_csv('./../data/prepared/sales_grouped.csv', ',')
sales_grouped.head()

,ZIP CODE,GROSS SQUARE FEET,SALE PRICE,PRICE PER GSQ
0,11201,3613.533333,6.529603e+06,1902.470606
1,11203,1763.665217,6.150363e+05,366.587064
2,11204,2059.094595,1.172006e+06,615.601979
3,11205,2947.439024,2.079976e+06,743.597238
4,11206,2761.061224,1.780433e+06,653.258047


Now, filter out all ZIP entries which are not in the sales (-> not in Brooklyn). Otherwise Folium/Colorpleth gets into troubles later 

In [51]:
import geopandas as gpd

all_zip_codes = gpd.read_file('./../data/intermediate/ZipCodeBoundaries.geojson')
all_zip_codes['ZIPCODE'] = pd.to_numeric(all_zip_codes['ZIPCODE'])
brooklyn_zip_codes = all_zip_codes[(all_zip_codes["ZIPCODE"].isin(sales_grouped['ZIP CODE']))].copy()
brooklyn_zip_codes.to_file('./../data/prepared/ZipCodeBoundaries_BrooklynOnly.geojson', driver='GeoJSON')
brooklyn_zip_codes.head(3)

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."


In [48]:
import folium

m = folium.Map(
    location=[40.643063306000045, -73.89848712999998],
    tiles=None,
    zoom_start=10
)

folium.TileLayer("Stamen Toner", name="Stamen Toner").add_to(m)
folium.TileLayer("Stamen Terrain", name="Stamen Terrain").add_to(m)
folium.TileLayer("Open Street Map", name="Open Street Map").add_to(m)

folium.Choropleth(
    name="average price per gross square feet",
    geo_data="data/intermediate/ZipCodeBoundaries_BrooklynOnly.geojson",
    data=sales,
    columns=["ZIP CODE", "PRICE PER GSQ"],
    key_on="feature.properties.ZIPCODE",
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Price per gross sq ($)'
).add_to(m)
folium.LayerControl().add_to(m)
m

FileNotFoundError: [Errno 2] No such file or directory: 'data/intermediate/ZipCodeBoundaries_BrooklynOnly.geojson'

https://github.com/geopandas/geopandas/issues/830